### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="tcp-ecn" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "router", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "delay", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
 {'name': "rx0",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils']}, 
]
net_conf = [
 {"name": "net-tx", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx0",   "addr": "10.0.0.100"}, {"name": "delay", "addr": "10.0.0.2"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.2"}, {"name": "router", "addr": "10.0.2.1"}]},
 {"name": "net-rx", "subnet": "10.0.5.0/24", "nodes": [{"name": "router",   "addr": "10.0.5.1"}, {"name": "rx0", "addr": "10.0.5.100"},]}

]
route_conf = [
 {"addr": "10.0.5.0/24", "gw": "10.0.0.2", "nodes": ["tx0"]}, 
 {"addr": "10.0.5.0/24", "gw": "10.0.2.1", "nodes": ["delay"]},

 {"addr": "10.0.0.0/24", "gw": "10.0.5.1", "nodes": ["rx0"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.2", "nodes": ["router"]}

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure Experiment

In [ ]:
for node in slice.get_nodes():
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
#inital configuration for the senders
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=cubic")
slice.get_node(name="tx0").execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

In [ ]:
#configuration for DUALPI2 bottleneck
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git
cd iproute2
sudo chmod +x configure
sudo ./configure
sudo make
sudo make install"""
slice.get_node(name="router").execute(cmd_dualpi2)
slice.get_node(name="router").execute("sudo modprobe sch_dualpi2")

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute Experiment

In [ ]:
# nodes and instances

tx0_node = slice.get_node(name="tx0")
rx0_node = slice.get_node(name="rx0")
delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router")

# interfaces

tx0_egress_iface  = tx0_node.get_interface(network_name = "net-tx")

delay_ingress_tx_iface  = delay_node.get_interface(network_name = "net-tx")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx_name = delay_ingress_tx_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-delay-router")
router_egress_iface  = router_node.get_interface(network_name = "net-rx")
router_egress_name  = router_egress_iface.get_device_name()


rx0_ingress_iface  = rx0_node.get_interface(network_name = "net-rx")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [2],  # n x bandwidth delay product
    'btl_capacity': [100], #in Mbps #'btl_capacity': [100, 1000]
    'base_rtt': [25], # in ms #'base_rtt': [5, 10, 50, 100],
    'aqm': ['single_queue_FQ'], #single_queue_FQ
    'ecn_threshold': [1, 20], # in ms #'ecn_threshold': [1, 5, 20]
    'ecn_support': [0, 1, 2],  # 0: noecn, 1: ecn, 2: accecn #'rx0_ecn': [0, 1, 2] #it will apply to both the sender and the receiver
    'cc_tx0': ["cubic","prague"],
    'trial': [1] #'trial': [1, 2, 3, 4, 5]
}

factor_names = list(exp_factors.keys())
factor_lists = itertools.product(*exp_factors.values())

flow_number_tx0=1 #number of tx0 flows

exp_lists = [
    {k: v for k, v in zip(factor_names, fl) 
     if (k != 'ecn_threshold') or 
        (fl[factor_names.index('aqm')] != 'FIFO' and fl[factor_names.index('ecn_support')] != 0)}
    for fl in factor_lists
    if ((fl[factor_names.index('n_bdp')] * fl[factor_names.index('base_rtt')] >= fl[factor_names.index('ecn_threshold')]
        or (fl[factor_names.index('aqm')] != 'FIFO' and fl[factor_names.index('ecn_support')] != 0))
        and not (fl[factor_names.index('cc_tx0')] == 'prague' and 
                 not (fl[factor_names.index('ecn_threshold')] == 1 and fl[factor_names.index('ecn_support')] == 2))
        and not (fl[factor_names.index('cc_tx0')] == 'cubic' and 
                 fl[factor_names.index('ecn_support')] == 2))
]

# Remove duplicates
exp_lists = [dict(t) for t in {frozenset(item.items()) for item in exp_lists}]

data_dir_tx0 = slice_name + 'singlebottleneck'+"-tx0"

print("Number of experiments:",len(exp_lists))

In [ ]:
# run experiments
import time
d = 20 #duration in seconds

em = [delay_ingress_tx_name, delay_egress_name]

commands_noecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=0''' 

commands_ecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=cubic  
sudo sysctl -w net.ipv4.tcp_ecn=1'''

commands_accecn='''
sudo sysctl -w net.ipv4.tcp_congestion_control=prague  
sudo sysctl -w net.ipv4.tcp_ecn=3''' 

ecn_list=[commands_noecn, commands_ecn, commands_accecn]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_support'], exp['trial'])
    
    file_out_tx0_json = name_tx0+"-result.json"
    file_out_tx0_ss = name_tx0+"-ss.txt"
    stdout_tx0_json, stderr_tx0_json = tx0_node.execute("ls " + file_out_tx0_json, quiet=True) 
    stdout_tx0_ss, stderr_tx0_ss = tx0_node.execute("ls " + file_out_tx0_ss, quiet=True)
    
    if len(stdout_tx0_json) and len(stdout_tx0_ss):
        print("Already have " + name_tx0 + ", skipping")

    elif len(stderr_tx0_json) or len(stderr_tx0_ss):
        print("Running experiment to generate " + name_tx0) 
        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
         
        #receiver ecn configuration
        rx0_node.execute(ecn_list[exp['ecn_support']])
        
        #sender ecn configuration
        tx0_node.execute(ecn_list[exp['ecn_support']])
        
        #aqm type selection
        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
        cmds_specific = {
        'FIFO': "sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer}".format(iface=router_egress_name, buffer=btl_limit),
        'single_queue_FQ': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, threshold=exp.get('ecn_threshold', 100)),
        'Codel': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'FQ': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_Codel': "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(iface=router_egress_name, packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'DualPI2': "sudo tc qdisc add dev {iface} parent 1:3 handle 3: dualpi2 target {threshold}ms".format(iface=router_egress_name, threshold=exp.get('ecn_threshold', 0))
        }

        router_node.execute(cmds_prefix)
        router_node.execute(cmds_specific[ exp['aqm'] ])
            
        rx0_node.execute("killall iperf3")
        
        ss_tx0_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
        
        rx0_node.execute("iperf3 -s -1 -p 4000 -D")
        
        tx0_node.execute_thread(ss_tx0_script.format(flow=name_tx0, duration=d))
        stdout, stderr =tx0_node.execute("sleep 1; iperf3 -c 10.0.5.100 -t {duration} -P {flows} -C {cc} -p 4000 -J > {flow}-result.json".format(flow =name_tx0, duration=d, flows=flow_number_tx0, cc=exp['cc_tx0']))
        time.sleep(3)  
        
print("finished")
        
        

### Analysis of the results

In [ ]:
for exp in exp_lists:
    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_support'], exp['trial'])
    
    
    file_out_tx0_csv = name_tx0+"-ss.csv"
    stdout_tx0_csv, stderr_tx0_csv = tx0_node.execute("ls " + file_out_tx0_csv, quiet=True) 

    if len(stdout_tx0_csv):
        print("Already have " + name_tx0 + ", skipping")

    elif len(stderr_tx0_csv):
        print("Running to generate csv files " + name_tx0)
    
        ss_tx0_script_processing="""

        f_1={types}; 
        rm -f ${{f_1}}-ss.csv;
        cat ${{f_1}}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT"> ${{f_1}}-ss-processed.txt; 
        cat ${{f_1}}-ss-processed.txt | awk '{{print $1}}' > ts-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\bcwnd:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\brtt:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1   > srtt-${{f_1}}.txt; 
        cat ${{f_1}}-ss-processed.txt | grep -oP '\\bfd=.*?(\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-${{f_1}}.txt;
        paste ts-${{f_1}}.txt fd-${{f_1}}.txt cwnd-${{f_1}}.txt srtt-${{f_1}}.txt -d ',' > ${{f_1}}-ss.csv;""".format(types=name_tx0)
     
        tx0_node.execute(ss_tx0_script_processing)

tx0_node.execute('mkdir '+data_dir_tx0)

tx0_node.execute('mv *.json '+ data_dir_tx0)
tx0_node.execute('mv *.txt '+ data_dir_tx0)
tx0_node.execute('mv *.csv '+ data_dir_tx0)

tx0_node.execute('tar -czvf '+data_dir_tx0+ '.tgz ' +  data_dir_tx0)

In [ ]:
slice_name_str = repr(slice_name)

content_tx0 = f"""
# generate full factorial experiment
import itertools
import json
import pandas as pd

exp_lists = {exp_lists}
slice_name = {slice_name_str}

data_dir_tx0 = slice_name + 'singlebottleneck' + "-tx0"

throughput_data = {{}}  # Initialize the dictionary
srtt_data = {{}}
cwnd_data= pd.DataFrame()
srtt_data_time= pd.DataFrame()
for exp in exp_lists:
    name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_support'], exp['trial'])

    # Load the JSON output file into a Python object
    with open(f"/home/ubuntu/{{data_dir_tx0}}/{{name_tx0}}-result.json") as f:
        iperf3_data = json.load(f)

    throughput_data[name_tx0] = iperf3_data['end']['sum_received']['bits_per_second'] / (1000000 * 1)  # to convert Mbit

    # Average SRTT for Each Flow
    columns = ['timestamp', 'flow ID', 'cwnd', 'srtt']
    df_f1 = pd.read_csv(f"/home/ubuntu/{{data_dir_tx0}}/{{name_tx0}}-ss.csv", names=columns)
    
    # Filter out rows with flow ID = 4, they are for the control flows
    df_f1 = df_f1[df_f1['flow ID'] != 4].reset_index(drop=True)

    average_RTT_f1 = df_f1['srtt'].mean()
    
    cwnd_data[name_tx0] = df_f1['cwnd']
    srtt_data[name_tx0] = average_RTT_f1
    srtt_data_time[name_tx0] = df_f1['srtt']

# Save throughput_data to a JSON file
with open('throughput_data.json', 'w') as f:
    json.dump(throughput_data, f)

# Save srtt_data to a JSON file
with open('srtt_data.json', 'w') as f:
    json.dump(srtt_data, f)

cwnd_data.to_csv("consolidated_cwnd_data.csv", index=False)
srtt_data_time.to_csv("time_srtt.csv", index=False)

"""

tx0_file_path = 'analysis_tx0.py'

# Write the content to the new file
with open(tx0_file_path, 'w') as new_file:
    new_file.write(content_tx0)

print(f"Content written to {tx0_file_path}")

In [ ]:
cmds_py_install = '''
            sudo apt-get -y install python3
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''

tx0_node.execute(cmds_py_install)

In [ ]:
import os
tx0_node.upload_file(os.path.join (os.getcwd(), "analysis_tx0.py"),"/home/ubuntu/analysis_tx0.py")

In [ ]:
tx0_node.execute('python3 analysis_tx0.py')

In [ ]:
tx0_node.download_file("/home/fabric/work/tput_tx0.json","/home/ubuntu/throughput_data.json")
tx0_node.download_file("/home/fabric/work/srtt_tx0.json","/home/ubuntu/srtt_data.json")
tx0_node.download_file("/home/fabric/work/cwnd_tx0.csv","/home/ubuntu/consolidated_cwnd_data.csv")

In [ ]:
import json
import os

# Initialize empty variables
throughput_data = {}
srtt_data = {}

# Directory containing JSON files
data_directory = '/home/fabric/work/'

# List of JSON files in the directory
json_files = [f for f in os.listdir(data_directory) if f.endswith('.json')]

# Load data from each JSON file and update the variables
for file_name in json_files:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Check if the file contains throughput data or srtt data based on its name
    if 'tput' in file_name:
        throughput_data.update(data)
    elif 'srtt' in file_name:
        srtt_data.update(data)

In [ ]:
throughput_data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

btl_cap=int(1000*exp_factors['btl_capacity'][0]*exp_factors['base_rtt'][0]/8)
btl_limit_noecn=int(1000*exp_factors['btl_capacity'][0]*exp_factors['base_rtt'][0]*(exp_factors['n_bdp'][0]+1)/8)
btl_limit_20=int(1000*exp_factors['btl_capacity'][0]*(exp_factors['base_rtt'][0]+20)/8)
btl_limit_1=int(1000*exp_factors['btl_capacity'][0]*(exp_factors['base_rtt'][0]+1)/8)

packet_limits=[int(btl_limit_noecn/1500)+1, int(btl_limit_20/1500)+1,int(btl_limit_1/1500)+1, int(btl_limit_1/1500)+1]

bottleneck_limit=int(btl_limit_noecn/1500)+1
bottleneck_limit_cap=int(btl_cap/1500)+1

df = pd.read_csv('/home/fabric/work/cwnd_tx0.csv')
custom_order = ['cubic_2.0_100_25_single_queue_FQ_none_0_1', 'cubic_2.0_100_25_single_queue_FQ_20_1_1', 'cubic_2.0_100_25_single_queue_FQ_1_1_1','prague_2.0_100_25_single_queue_FQ_1_2_1']
titles=['CUBIC - No ECN', 'CUBIC - 20 ms ECN Threshold' , 'CUBIC - 1ms ECN Threshold','Prague - 1ms ECN Threshold']
ecn_limits=[int(btl_limit_noecn/1500)+1, int(btl_limit_20/1500)+1,int(btl_limit_1/1500)+1]
assert set(custom_order) == set(df.columns)
num_columns = len(df.columns)

global_min_cwnd = min(df[custom_order].min())
global_max_cwnd = max(df[custom_order].max())


fig, axs = plt.subplots(1, num_columns, figsize=(5 * num_columns+6, 7))  # Adjust the figure size as needed

y_limits = (global_min_cwnd, global_max_cwnd+10)

# Iterate over the columns and create a subplot for each
for i, column in enumerate(custom_order):
    axs[i].plot(df.index, df[column])
    axs[i].set_title(f"{titles[i]} - Average Utilization: {format(throughput_data[column], '.2f')} %", fontsize=14)
    axs[i].set_xlabel('Index')
    axs[i].set_ylim(y_limits)
    axs[i].set_ylabel('CWND')
    axs[i].axhline(y=packet_limits[i], color='r', linestyle='--', label=f'ECN Limit')
    axs[i].axhline(y=bottleneck_limit, color='b', linestyle='--', label=f'Buffer Size')
    axs[i].axhline(y=bottleneck_limit_cap, color='g', linestyle='--', label=f'Bottleneck Capacity')
    axs[i].legend()  # Display the legend

plt.tight_layout()  # Adjust subplots to fit into the figure area.
fig.suptitle('100 Mbps Bottleneck Capacity -  2BDP Buffer Size - 25ms Base RTT - Single Queue FQ', y=1.05,fontsize=18)
plt.savefig('plots.png', dpi=300)  # Adjust the file name and DPI as needed
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assume exp_factors_list is a list of dictionaries where each dictionary represents a set of parameters for an experiment
def generate_column_names_and_limits(exp_lists):
    column_names = []
    limits = []
    for exp in exp_lists:
        name_tx0="%s_%0.1f_%d_%d_%s_%s_%d_%d" % (exp['cc_tx0'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_support'], exp['trial'])
        column_names.append(name_tx0)

        # Compute the limits
        btl_cap = int(1000 * exp['btl_capacity'] * exp['base_rtt'] / 8)
        btl_limit_noecn = int(btl_cap * (exp['n_bdp'] + 1))
        ecn_threshold = exp.get('ecn_threshold', 0)
        btl_limit_ecn = int(1000 * exp['btl_capacity'] * (exp['base_rtt'] + ecn_threshold) / 8)

        packet_limit_ecn = int(btl_limit_ecn / 1500) + 1
        bottleneck_limit = int(btl_limit_noecn / 1500) + 1
        bottleneck_capacity_limit = int(btl_cap / 1500) + 1

        limits.append((packet_limit_ecn, bottleneck_limit, bottleneck_capacity_limit))

    return column_names, limits


def generate_dynamic_titles(specified_params, avg_utilization):
    included_keys = {'cc_tx0','ecn_threshold','ecn_support'}
    excluded_keys = {'btl_capacity', 'base_rtt', 'n_bdp','aqm'}

    title_map = {
        'cc_tx0': lambda: f"{specified_params['cc_tx0']}",
        'btl_capacity': lambda: f"{specified_params['btl_capacity']} Mbps",
        'n_bdp': lambda: f"{specified_params['n_bdp']} BDP",
        'base_rtt': lambda: f"{specified_params['base_rtt']} ms RTT",
        'ecn_threshold': lambda: f"ECN Threshold={specified_params['ecn_threshold']} ms",
        'ecn_support': lambda: f"ECN Support={specified_params['ecn_support']}",
        'aqm': lambda: f"AQM Type={specified_params['aqm']}"
    }

    # Generate the main title with included keys
    main_title = ' - '.join(val() for key, val in title_map.items() if key in included_keys and specified_params.get(key) is not None)
    main_title += f" - Average Utilization: {avg_utilization:.2f} %"

    # Generate the additional title with excluded keys
    sup_title = ' - '.join(val() for key, val in title_map.items() if key in excluded_keys and specified_params.get(key) is not None)

    return main_title, sup_title

column_names, limits = generate_column_names_and_limits(exp_lists)

# Read the data
df = pd.read_csv('/home/fabric/work/cwnd_tx0.csv')

num_columns = len(column_names)
fig, axs = plt.subplots(1, num_columns, figsize=(5 * num_columns + 15, 7))

if num_columns == 1:
    axs = [axs]  # Make it a list to keep the indexing consistent

for i, column in enumerate(column_names):
    avg_utilization = throughput_data[column]  # Get the average utilization for this column
    dynamic_title, sup_title = generate_dynamic_titles(exp_lists[i], avg_utilization)
    axs[i].plot(df.index, df[column])
    axs[i].set_title(dynamic_title, fontsize=14)
    axs[i].set_xlabel('Index')
    axs[i].set_ylabel('CWND')
    axs[i].axhline(y=limits[i][0], color='r', linestyle='--', label='ECN Limit')
    axs[i].axhline(y=limits[i][1], color='b', linestyle='--', label='Buffer Size')
    axs[i].axhline(y=limits[i][2], color='g', linestyle='--', label='Bottleneck Capacity')
    axs[i].legend()

plt.tight_layout()
plt.suptitle(sup_title, y=1.05,fontsize=18) 

plt.show()